# Bag of words model
Final Project for NLP

## Loading data and preprocessing

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

In [2]:
idA = {1:'American (Traditional)', 2:'Italian',3:'Mexican',4:'Chinese',5:'American (New)',6:'Japanese', 0:'Other'}
idB = {1:'Nightlife', 2:'Fast Food', 3:'Breakfast & Brunch', 4:'Coffee & Tea', 5:'Cafes', 0:'Other'}
df = pd.read_pickle("typeAB_not_0.pickle")
df_full = pd.read_pickle("filtered.pickle")

In [36]:
mini_df = df[:10000]
print(len(df))

921172


In [8]:
review = mini_df['review']
y = mini_df['stars']

In [9]:
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
def sentence_tokenizer(s):
    """sentence tokenizer --modified from EDA version
    :parameter 
        s: String 
    :return
        tokenized_s: String"""
    s = s.lower()
    tokenized_s = nltk.word_tokenize(s)
    english_stopwords = set(stopwords.words('english'))
    tokenized_s = [w for w in tokenized_s if w not in english_stopwords if w.isalpha()]
    tokenized_s = " ".join(tokenized_s)
    return tokenized_s

In [30]:
%%time
X = []
for i in tqdm(range(len(review))):
    X.append(sentence_tokenizer(review[i]))

100%|███████████████████████████████████| 10000/10000 [00:34<00:00, 286.45it/s]


Wall time: 34.9 s


In [54]:
processed_df = pd.DataFrame({'review':X,'star':y})
processed_df.to_csv('processes_mini.csv')

In [17]:
from sklearn.model_selection import train_test_split
X_train, X_test,y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

## Bag of words model

### Count Vectorizer
https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html

In [34]:
from sklearn.feature_extraction.text import CountVectorizer
count_vectorizer = CountVectorizer(analyzer='word', max_features=5000)  #can use n_gram, see document
X_train_features = count_vectorizer.fit_transform(X_train)
X_train_features = X_train_features.toarray()
X_test_features = count_vectorizer.transform(X_test)
X_test_features = X_test_features.toarray()

In [35]:
%%time
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier(n_estimators = 100, n_jobs=-1, random_state=42) 
forest = forest.fit( X_train_features, y_train )

Wall time: 41 s


## Test and evalutaion

Need to choose a metric

In [33]:
model = forest

In [37]:
y_pred = model.predict(X_test_features)

In [43]:
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
print('F1 score',f1_score(y_test, y_pred, average='macro'))
print('Accuracy', accuracy_score(y_test, y_pred))

F1 score 0.3669593372208219
Accuracy 0.548
